# Characterizing CpG Methylation (5x data)

In this notebook, general methylation landscapes in *Montipora capitata* and *Pocillopora acuta* will be characterized based on WGSB, RRBS, and MBD-BSseq data. I will also assess CG motif overlaps with various genome feature tracks to understand where methylation may occur across the genome. I will use 5x data.

1. Characterize overlap between CG motifs and genome feature tracks
1. Download coverage files
2. Characterize methylation for each CpG dinucleotide
3. Characterize genomic locations of all sequenced data, methylated CpGs, sparsely methylated CpGs, and unmethylated CpGs for each sequencing type

## 0. Set working directory and obtain checksums

In [1]:
!pwd

/Users/yaamini/Documents/Meth_Compare/scripts


In [2]:
cd ../analyses/

/Users/yaamini/Documents/Meth_Compare/analyses


In [3]:
#!mkdir Characterizing-CpG-Methylation-5x

In [4]:
cd Characterizing-CpG-Methylation-5x/

/Users/yaamini/Documents/Meth_Compare/analyses/Characterizing-CpG-Methylation-5x


In [5]:
!wget https://gannet.fish.washington.edu/metacarcinus/FROGER_meth_compare/20200410/all_031520-TG-bs_files_GANNET_md5sum.txt

--2020-06-11 11:42:02--  https://gannet.fish.washington.edu/metacarcinus/FROGER_meth_compare/20200410/all_031520-TG-bs_files_GANNET_md5sum.txt
Resolving gannet.fish.washington.edu (gannet.fish.washington.edu)... 128.95.149.52
Connecting to gannet.fish.washington.edu (gannet.fish.washington.edu)|128.95.149.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90413 (88K) [text/plain]
Saving to: ‘all_031520-TG-bs_files_GANNET_md5sum.txt.1’

all_031520-TG-bs_fi 100%[===================>]  88.29K  --.-KB/s    in 0.001s  

2020-06-11 11:42:02 (65.4 MB/s) - ‘all_031520-TG-bs_files_GANNET_md5sum.txt.1’ saved [90413/90413]



In [6]:
!head all_031520-TG-bs_files_GANNET_md5sum.txt

04829778554df5986ae415fcda3b7e81  /Volumes/web/seashell/bu-mox/scrubbed/031520-TG-bs/Meth9_R1_001_val_1.fq.gz
e1048fea898bc32cb03ff801534183d9  /Volumes/web/seashell/bu-mox/scrubbed/031520-TG-bs/Meth15_R2_001_val_2.fq.gz
d6e026bb59b10a11ad9b51b8acdd18a7  /Volumes/web/seashell/bu-mox/scrubbed/031520-TG-bs/Meth5_R2_001_val_2.fq.gz
bfe70cae27f3251ead4e6686391940ca  /Volumes/web/seashell/bu-mox/scrubbed/031520-TG-bs/Meth8_R1_001_val_1.fq.gz_G_to_A.fastq
26c6f90dd9cef5e30f32e312007f3176  /Volumes/web/seashell/bu-mox/scrubbed/031520-TG-bs/Meth15_R2_001_val_2.fq.gz_G_to_A.fastq
f41790ce58777f20ee742cba75692065  /Volumes/web/seashell/bu-mox/scrubbed/031520-TG-bs/Meth1_R1_001_val_1.fq.gz
4ed014c23ba4c28681d5b4af17e95346  /Volumes/web/seashell/bu-mox/scrubbed/031520-TG-bs/Meth14_R1_001_val_1.fq.gz
fc3ad5f9624c63e28bab515b5848158c  /Volumes/web/seashell/bu-mox/scrubbed/031520-TG-bs/Meth13_R2_001_val_2.fq.gz_C_to_T.fastq
8b2c14989c4638fa2cdd7d16a36a7b99  /Volumes/web/seashell/bu-mox/scrubbed/03152

### *M. capitata*

In [7]:
#Get all lines from original checksum document
#Extract information for 5x bedgraphs
#Extract information for Mcap data only
#Only keep the first 32 characters in each line (md5sum hashes)
#Save hashes
!cat all_031520-TG-bs_files_GANNET_md5sum.txt \
| grep 5x.bedgraph \
| grep Mcap \
| cut -c1-32 \
> Mcap-5xbedgraph-GANNET-md5sum-hashes.txt

In [8]:
#Get all lines from original checksum document
#Extract information for 5x bedgraphs
#Extract information for Mcap data only
#Reverse order of characters in each line
#Only keep the first 48 characters in each line
#actually the last 48 characters in the original file, which maps to paths locally
#Reverse characters
#Save paths
!cat all_031520-TG-bs_files_GANNET_md5sum.txt \
| grep 5x.bedgraph \
| grep Mcap \
| rev \
| cut -c1-47 \
| rev \
> Mcap-5xbedgraph-GANNET-md5sum-paths.txt

In [9]:
#Paste hashes and paths to create a md5sum file
#Save checksum file
#Check output
#Count number of lines        
!paste Mcap-5xbedgraph-GANNET-md5sum-hashes.txt Mcap-5xbedgraph-GANNET-md5sum-paths.txt \
> Mcap-5xbedgraph-GANNET-md5sum.txt
!head Mcap-5xbedgraph-GANNET-md5sum.txt
!wc -l Mcap-5xbedgraph-GANNET-md5sum.txt

04fb72d5df60656e6cec15637164fbec	Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
b2f097299df0cb7d518d22338fdcf39f	Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
073d1c40116a3f93f7a7022cfb4cd3d2	Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
83035e7e47b8ad486de22dacc17ae8ed	Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
a255210553db073e5458ccb523a34798	Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
6493359aad0b4228f65b5e563d337ceb	Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
fc0f66cf04ffebe76d61c1db75cfed6e	Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
1d7c24b238dc72cd92346213b3523611	Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
2bb476cb98072f0e76bfb5c318246c38	Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
       9 Mcap-5xbedgraph-GANNET-md5sum.txt


### *P. acuta*

In [10]:
#Get all lines from original checksum document
#Extract information for 5x bedgraphs
#Extract information for Pact data only
#Only keep the first 32 characters in each line (md5sum hashes)
#Save hashes
!cat all_031520-TG-bs_files_GANNET_md5sum.txt \
| grep 5x.bedgraph \
| grep Pact \
| cut -c1-32 \
> Pact-5xbedgraph-GANNET-md5sum-hashes.txt

In [11]:
#Get all lines from original checksum document
#Extract information for 5x bedgraphs
#Extract information for Pact data only
#Reverse order of characters in each line
#Only keep the first 48 characters in each line
#actually the last 48 characters in the original file, which maps to paths locally
#Reverse characters
#Save paths
!cat all_031520-TG-bs_files_GANNET_md5sum.txt \
| grep 5x.bedgraph \
| grep Pact \
| rev \
| cut -c1-46 \
| rev \
> Pact-5xbedgraph-GANNET-md5sum-paths.txt

In [12]:
#Paste hashes and paths to create a md5sum file
#Save checksum file
#Check output
#Count number of lines        
!paste Pact-5xbedgraph-GANNET-md5sum-hashes.txt Pact-5xbedgraph-GANNET-md5sum-paths.txt \
> Pact-5xbedgraph-GANNET-md5sum.txt
!head Pact-5xbedgraph-GANNET-md5sum.txt
!wc -l Pact-5xbedgraph-GANNET-md5sum.txt

c838562956c7abe3656a2b7438a40dc1	Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
c9a4b002113e2501d81e4762cf952b79	Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
1ec934f5b4ce012b64b77dd69d70ee5f	Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
c456156b7f6a11543d8dc697e8e74b4e	Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
d634ffc3f062d248e36b8dddc9a315e0	Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
a2b842c439c3df3fb699690cd5b55d5a	Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
5994ba73d412d8992f2465b148f5ae80	Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
ed4428a6c8cb6a4964687d91c0d8ccb3	Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
736fd3802ce1b45b6eb32abf6e1bcb3f	Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
       9 Pact-5xbedgraph-GANNET-md5sum.txt


## *M. capitata*

In [13]:
#Make a directory for Mcap output
#!mkdir Mcap

In [14]:
cd Mcap/

/Users/yaamini/Documents/Meth_Compare/analyses/Characterizing-CpG-Methylation-5x/Mcap


### 1. Characterize CG motif locations in feature tracks

#### 1a. Set variable paths

In [15]:
bedtoolsDirectory = "/usr/local/bin/"

In [16]:
mcGenes = "../../../genome-feature-files/Mcap.GFFannotation.gene.gff"

In [17]:
mcCDS = "../../../genome-feature-files/Mcap.GFFannotation.CDS.gff"

In [18]:
mcIntron = "../../../genome-feature-files/Mcap.GFFannotation.intron.gff"

In [19]:
mcFlanks = "../../../genome-feature-files/Mcap.GFFannotation.flanks.gff"

In [20]:
mcUpstream = "../../../genome-feature-files/Mcap.GFFannotation.flanks.Upstream.gff"

In [21]:
mcDownstream = "../../../genome-feature-files/Mcap.GFFannotation.flanks.Downstream.gff"

In [22]:
mcIntergenic = "../../../genome-feature-files/Mcap.GFFannotation.intergenic.bed"

In [23]:
mcCGMotifs = "../../../genome-feature-files/Mcap_CpG.gff"

#### 1b. Check variable paths

In [24]:
!head {mcGenes}
!wc -l {mcGenes}

1	AUGUSTUS	gene	18387	18755	0.97	-	.	g21532
1	AUGUSTUS	gene	22321	27293	0.23	-	.	g21533
1	AUGUSTUS	gene	37447	52266	1	+	.	g21534
1	AUGUSTUS	gene	58322	62557	1	-	.	g21535
1	AUGUSTUS	gene	64466	84798	1	+	.	g21536
1	AUGUSTUS	gene	88347	97184	1	+	.	g21537
1	AUGUSTUS	gene	100215	109729	0.99	-	.	g21538
1	AUGUSTUS	gene	109867	128510	0.89	+	.	g21539
1	AUGUSTUS	gene	132854	139285	1	-	.	g21540
1	AUGUSTUS	gene	148344	149588	0.44	+	.	g21541
   53875 ../../../genome-feature-files/Mcap.GFFannotation.gene.gff


In [25]:
!head {mcCDS}
!wc -l {mcCDS}

1	AUGUSTUS	CDS	18387	18755	0.97	-	0	transcript_id "g21532.t1"; gene_id "g21532";
1	AUGUSTUS	CDS	22321	22608	0.55	-	0	transcript_id "g21533.t1"; gene_id "g21533";
1	AUGUSTUS	CDS	26301	27293	0.29	-	0	transcript_id "g21533.t1"; gene_id "g21533";
1	AUGUSTUS	CDS	37447	37810	1	+	0	transcript_id "g21534.t1"; gene_id "g21534";
1	AUGUSTUS	CDS	45038	45208	1	+	2	transcript_id "g21534.t1"; gene_id "g21534";
1	AUGUSTUS	CDS	46625	47272	1	+	2	transcript_id "g21534.t1"; gene_id "g21534";
1	AUGUSTUS	CDS	49943	50132	1	+	2	transcript_id "g21534.t1"; gene_id "g21534";
1	AUGUSTUS	CDS	51903	52266	1	+	1	transcript_id "g21534.t1"; gene_id "g21534";
1	AUGUSTUS	CDS	58322	59506	1	-	0	transcript_id "g21535.t1"; gene_id "g21535";
1	AUGUSTUS	CDS	62261	62557	1	-	0	transcript_id "g21535.t1"; gene_id "g21535";
  224096 ../../../genome-feature-files/Mcap.GFFannotation.CDS.gff


In [26]:
!head {mcIntron}
!wc -l {mcIntron}

1	AUGUSTUS	intron	22609	26300	0.25	-	.	transcript_id "g21533.t1"; gene_id "g21533";
1	AUGUSTUS	intron	37811	45037	1	+	.	transcript_id "g21534.t1"; gene_id "g21534";
1	AUGUSTUS	intron	45209	46624	1	+	.	transcript_id "g21534.t1"; gene_id "g21534";
1	AUGUSTUS	intron	47273	49942	1	+	.	transcript_id "g21534.t1"; gene_id "g21534";
1	AUGUSTUS	intron	50133	51902	1	+	.	transcript_id "g21534.t1"; gene_id "g21534";
1	AUGUSTUS	intron	59507	62260	1	-	.	transcript_id "g21535.t1"; gene_id "g21535";
1	AUGUSTUS	intron	64578	64654	1	+	.	transcript_id "g21536.t1"; gene_id "g21536";
1	AUGUSTUS	intron	64735	67263	1	+	.	transcript_id "g21536.t1"; gene_id "g21536";
1	AUGUSTUS	intron	67319	71345	1	+	.	transcript_id "g21536.t1"; gene_id "g21536";
1	AUGUSTUS	intron	71456	72865	1	+	.	transcript_id "g21536.t1"; gene_id "g21536";
  170950 ../../../genome-feature-files/Mcap.GFFannotation.intron.gff


In [27]:
!head {mcFlanks}
!wc -l {mcFlanks}

1	AUGUSTUS	gene	17387	18386	0.97	-	.	g21532
1	AUGUSTUS	gene	18756	19755	0.97	-	.	g21532
1	AUGUSTUS	gene	21321	22320	0.23	-	.	g21533
1	AUGUSTUS	gene	27294	28293	0.23	-	.	g21533
1	AUGUSTUS	gene	36447	37446	1	+	.	g21534
1	AUGUSTUS	gene	52267	53266	1	+	.	g21534
1	AUGUSTUS	gene	57322	58321	1	-	.	g21535
1	AUGUSTUS	gene	62558	63557	1	-	.	g21535
1	AUGUSTUS	gene	63466	64465	1	+	.	g21536
1	AUGUSTUS	gene	84799	85798	1	+	.	g21536
  114420 ../../../genome-feature-files/Mcap.GFFannotation.flanks.gff


In [28]:
!head {mcUpstream}
!wc -l {mcUpstream}

1	AUGUSTUS	gene	18756	19755	0.97	-	.	g21532
1	AUGUSTUS	gene	27294	28293	0.23	-	.	g21533
1	AUGUSTUS	gene	36447	37446	1	+	.	g21534
1	AUGUSTUS	gene	62558	63557	1	-	.	g21535
1	AUGUSTUS	gene	63466	64465	1	+	.	g21536
1	AUGUSTUS	gene	87347	88346	1	+	.	g21537
1	AUGUSTUS	gene	109730	109866	0.99	-	.	g21538
1	AUGUSTUS	gene	109730	109866	0.89	+	.	g21539
1	AUGUSTUS	gene	139286	140285	1	-	.	g21540
1	AUGUSTUS	gene	147344	148343	0.44	+	.	g21541
   57349 ../../../genome-feature-files/Mcap.GFFannotation.flanks.Upstream.gff


In [29]:
!head {mcDownstream}
!wc -l {mcDownstream}

1	AUGUSTUS	gene	17387	18386	0.97	-	.	g21532
1	AUGUSTUS	gene	21321	22320	0.23	-	.	g21533
1	AUGUSTUS	gene	52267	53266	1	+	.	g21534
1	AUGUSTUS	gene	57322	58321	1	-	.	g21535
1	AUGUSTUS	gene	84799	85798	1	+	.	g21536
1	AUGUSTUS	gene	97185	98184	1	+	.	g21537
1	AUGUSTUS	gene	99215	100214	0.99	-	.	g21538
1	AUGUSTUS	gene	128511	129510	0.89	+	.	g21539
1	AUGUSTUS	gene	131854	132853	1	-	.	g21540
1	AUGUSTUS	gene	149589	150588	0.44	+	.	g21541
   57406 ../../../genome-feature-files/Mcap.GFFannotation.flanks.Downstream.gff


In [30]:
!head {mcIntergenic}
!wc -l {mcIntergenic}

1	0	17386
1	19755	21320
1	28293	36446
1	53266	57321
1	85798	87346
1	98184	99214
1	129510	131853
1	140285	147343
1	150588	155443
1	158268	171840
   38845 ../../../genome-feature-files/Mcap.GFFannotation.intergenic.bed


In [31]:
!head {mcCGMotifs}
!wc -l {mcCGMotifs}

##gff-version 2.0
##date 2020-03-29
##Type DNA 1
1	fuzznuc	misc_feature	37	38	2.000	+	.	Sequence "1.1" ; note "*pat pattern1"
1	fuzznuc	misc_feature	90	91	2.000	+	.	Sequence "1.2" ; note "*pat pattern1"
1	fuzznuc	misc_feature	121	122	2.000	+	.	Sequence "1.3" ; note "*pat pattern1"
1	fuzznuc	misc_feature	132	133	2.000	+	.	Sequence "1.4" ; note "*pat pattern1"
1	fuzznuc	misc_feature	153	154	2.000	+	.	Sequence "1.5" ; note "*pat pattern1"
1	fuzznuc	misc_feature	170	171	2.000	+	.	Sequence "1.6" ; note "*pat pattern1"
1	fuzznuc	misc_feature	220	221	2.000	+	.	Sequence "1.7" ; note "*pat pattern1"
 28684519 ../../../genome-feature-files/Mcap_CpG.gff


#### 1c. Characterize overlaps with `bedtools`

In [32]:
!{bedtoolsDirectory}intersectBed -h


Tool:    bedtools intersect (aka intersectBed)
Version: v2.17.0
Summary: Report overlaps between two feature files.

Usage:   bedtools intersect [OPTIONS] -a <bed/gff/vcf> -b <bed/gff/vcf>

Options: 
	-abam	The A input file is in BAM format.  Output will be BAM as well.

	-ubam	Write uncompressed BAM output. Default writes compressed BAM.

	-bed	When using BAM input (-abam), write output as BED. The default
		is to write output in BAM when using -abam.

	-wa	Write the original entry in A for each overlap.

	-wb	Write the original entry in B for each overlap.
		- Useful for knowing _what_ A overlaps. Restricted by -f and -r.

	-loj	Perform a "left outer join". That is, for each feature in A
		report each overlap with B.  If no overlaps are found, 
		report a NULL feature for B.

	-wo	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlaps restricted by -f and -r.
		  Only A features with overlap are reported.

	-wao	Write the

In [33]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {mcCGMotifs} \
-b {mcGenes} \
> Mcap-CGMotif-Gene-Overlaps.txt

In [34]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {mcCGMotifs} \
-b {mcCDS} \
> Mcap-CGMotif-CDS-Overlaps.txt

In [35]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {mcCGMotifs} \
-b {mcIntron} \
> Mcap-CGMotif-Intron-Overlaps.txt

In [36]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {mcCGMotifs} \
-b {mcFlanks} \
> Mcap-CGMotif-Flanks-Overlaps.txt

In [37]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {mcCGMotifs} \
-b {mcUpstream} \
> Mcap-CGMotif-Flanks-Upstream-Overlaps.txt

In [38]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {mcCGMotifs} \
-b {mcDownstream} \
> Mcap-CGMotif-Flanks-Downstream-Overlaps.txt

In [39]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {mcCGMotifs} \
-b {mcIntergenic} \
> Mcap-CGMotif-Intergenic-Overlaps.txt

In [40]:
!wc -l *CGMotif* > Mcap-CGMotif-Overlaps-counts.txt

#### 1d. Summary

| *M. capitata* Genome Feature 	| Number individual features 	| **Overlaps with CG Motifs** 	| **% Total CG Motifs** 	|
|:----------------------------------:	|:------------------------------:	|:---------------------------:	|:--------------------:	|
|                Genes               	|             458273             	|           9450564          	|         32.9         	|
|          Coding Sequences          	|             283926             	|           1953206           	|          6.8         	|
|               Introns              	|             221428             	|           7503314          	|         26.2         	|
|               Flanking Regions              	|             114420             	|           2758953          	|         9.6         	|
|               Upstream Flanks              	|             57349             	|           1505544          	|         5.2         	|
|               Downstream Flanks              	|             57406             	|           1365975          	|         4.8         	|
|         Intergenic Regions         	|               38845              	|           16468305          	|         57.4         	|

### 2. Download coverage files

In [41]:
#Download Mcap WGBS and MBD-BS 5x sample bedgraphs
!wget -r -l1 --no-parent -A "*5x.bedgraph" https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Mcap_tg/dedup/

--2020-06-11 12:03:33--  https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Mcap_tg/dedup/
Resolving gannet.fish.washington.edu (gannet.fish.washington.edu)... 128.95.149.52
Connecting to gannet.fish.washington.edu (gannet.fish.washington.edu)|128.95.149.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Mcap_tg/dedup/index.html.tmp’

gannet.fish.washing     [ <=>                ]  42.27K  --.-KB/s    in 0.001s  

2020-06-11 12:03:35 (33.1 MB/s) - ‘gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Mcap_tg/dedup/index.html.tmp’ saved [43285]

Loading robots.txt; please ignore errors.
--2020-06-11 12:03:35--  https://gannet.fish.washington.edu/robots.txt
Reusing existing connection to gannet.fish.washington.edu:443.
HTTP request sent, awaiting response... 404 Not Found
2020-06-11 12:03:35 ERROR 404: Not Found.

Removing gannet

In [42]:
#Move samples from directory structure on gannet to cd
!mv gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Mcap_tg/dedup/* .

In [43]:
#Remove empty directory
!rm -r gannet.fish.washington.edu/

In [44]:
#Check downloaded files
!ls *bedgraph

Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph


In [45]:
#Download Mcap RRBS 5x sample bedgraphs
!wget -r -l1 --no-parent -A "*5x.bedgraph" https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Mcap_tg/nodedup/

--2020-06-11 12:03:45--  https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Mcap_tg/nodedup/
Resolving gannet.fish.washington.edu (gannet.fish.washington.edu)... 128.95.149.52
Connecting to gannet.fish.washington.edu (gannet.fish.washington.edu)|128.95.149.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Mcap_tg/nodedup/index.html.tmp’

gannet.fish.washing     [ <=>                ]  19.31K  --.-KB/s    in 0.001s  

2020-06-11 12:03:46 (27.2 MB/s) - ‘gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Mcap_tg/nodedup/index.html.tmp’ saved [19778]

Loading robots.txt; please ignore errors.
--2020-06-11 12:03:46--  https://gannet.fish.washington.edu/robots.txt
Reusing existing connection to gannet.fish.washington.edu:443.
HTTP request sent, awaiting response... 404 Not Found
2020-06-11 12:03:46 ERROR 404: Not Found.

Removing 

In [46]:
#Move samples from directory structure on gannet to cd
!mv gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Mcap_tg/nodedup/* .

In [47]:
#Remove empty directory
!rm -r gannet.fish.washington.edu/

In [48]:
!find *bedgraph

Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph


In [49]:
#Verify checksums from gannet
!md5sum -c ../Mcap-5xbedgraph-GANNET-md5sum.txt

Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK


In [50]:
!wc -l *bedgraph

 4571288 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 4661716 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 8791700 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 3173254 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 2648697 Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 3176517 Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
  583599 Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
  242390 Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
  153392 Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 28002553 total


In [51]:
!wc -l *bedgraph > Mcap-5x-bedgraph-counts.txt

### 3. Characterize methylation for each CpG dinucleotide

- Methylated: > 50% methylation
- Sparsely methylated: 10-50% methylation
- Unmethylated: < 10% methylation

##### Methylated loci

In [52]:
%%bash
for f in *bedgraph
do
    awk '{if ($4 >= 50) { print $1, $2, $3, $4 }}' ${f} \
    > ${f}-Meth
done

In [53]:
!head *Meth

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth <==
1 58745 58747 100.000000
1 103334 103336 100.000000
1 103347 103349 100.000000
1 103356 103358 100.000000
1 103360 103362 100.000000
1 103398 103400 100.000000
1 105953 105955 80.000000
1 106012 106014 50.000000
1 106155 106157 60.000000
1 106173 106175 66.666667

==> Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth <==
1 6905 6907 60.000000
1 7273 7275 80.000000
1 58745 58747 100.000000
1 59207 59209 100.000000
1 69235 69237 100.000000
1 69271 69273 80.000000
1 69275 69277 100.000000
1 69451 69453 100.000000
1 69580 69582 100.000000
1 69584 69586 100.000000

==> Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth <==
1 4948 4950 50.000000
1 4967 4969 50.000000
1 4986 4988 50.000000
1 57065 57067 80.000000
1 58609 58611 100.000000
1 58618 58620 100.000000
1 59207 59209 100.000000
1 59277 59279 100.000000
1 59393 59395 100.000000
1 59438 59440 100.000000

==> Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth <==
1 5861

In [54]:
!wc -l *-Meth

  450582 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  528902 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
 1059904 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  257741 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  184742 Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  231347 Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  106695 Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
   45506 Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
   29468 Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
 2894887 total


In [55]:
!wc -l *-Meth > Mcap-5x-Meth-counts.txt

##### Sparsely methylated loci

In [56]:
%%bash
for f in *bedgraph
do
    awk '{if ($4 < 50) { print $1, $2, $3, $4}}' ${f} \
    | awk '{if ($4 > 10) { print $1, $2, $3, $4 }}' \
    > ${f}-sparseMeth
done

In [57]:
!head *sparseMeth

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth <==
1 27782 27784 20.000000
1 80133 80135 20.000000
1 106202 106204 40.000000
1 140551 140553 33.333333
1 148080 148082 16.666667
1 150099 150101 40.000000
1 169735 169737 12.500000
1 169771 169773 42.857143
1 169796 169798 14.285714
1 169800 169802 16.666667

==> Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth <==
1 6550 6552 12.500000
1 6671 6673 20.000000
1 6996 6998 20.000000
1 7016 7018 40.000000
1 7019 7021 40.000000
1 7293 7295 16.666667
1 7427 7429 16.666667
1 74928 74930 14.285714
1 153767 153769 20.000000
1 193930 193932 20.000000

==> Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth <==
1 4190 4192 16.666667
1 4891 4893 33.333333
1 4910 4912 28.571429
1 4929 4931 33.333333
1 5005 5007 28.571429
1 5024 5026 40.000000
1 5151 5153 20.000000
1 5160 5162 16.666667
1 5228 5230 11.111111
1 6282 6284 11.111111

==> Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth <==
1 15092 15094 30.000000

In [58]:
!wc -l *sparseMeth

  547868 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  517805 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
 1000337 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  152042 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  135052 Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  179454 Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
   74839 Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
   28850 Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
   16793 Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
 2653040 total


In [59]:
!wc -l *-sparseMeth > Mcap-5x-sparseMeth-counts.txt

##### Unmethylated loci

In [60]:
%%bash
for f in *bedgraph
do
    awk '{if ($4 <= 10) { print $1, $2, $3, $4 }}' ${f} \
    > ${f}-unMeth
done

In [61]:
!head *unMeth

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth <==
1 6570 6572 0.000000
1 6713 6715 0.000000
1 6780 6782 0.000000
1 6813 6815 0.000000
1 6818 6820 0.000000
1 27606 27608 0.000000
1 27613 27615 0.000000
1 27641 27643 0.000000
1 27643 27645 0.000000
1 27674 27676 0.000000

==> Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth <==
1 4929 4931 0.000000
1 5665 5667 0.000000
1 6453 6455 0.000000
1 6484 6486 0.000000
1 6527 6529 0.000000
1 6570 6572 0.000000
1 6618 6620 0.000000
1 6652 6654 0.000000
1 6661 6663 0.000000
1 6668 6670 0.000000

==> Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth <==
1 4062 4064 0.000000
1 4069 4071 0.000000
1 4077 4079 0.000000
1 4086 4088 0.000000
1 4146 4148 0.000000
1 4150 4152 0.000000
1 4155 4157 0.000000
1 4172 4174 0.000000
1 4184 4186 0.000000
1 5043 5045 0.000000

==> Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth <==
1 3493 3495 0.000000
1 3518 3520 0.000000
1 3727 3729 0.000000
1 3752 3754 0.000000
1 3757 3759 0.000000


In [62]:
!wc -l *unMeth

 3572838 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 3615009 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 6731459 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 2763471 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 2328903 Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 2765716 Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
  402065 Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
  168034 Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
  107131 Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 22454626 total


In [63]:
!wc -l *-unMeth > Mcap-5x-unMeth-counts.txt

### 4. Characterize genomic locations of CpGs

#### 4a. Create BEDfiles

In [64]:
%%bash

for f in *bedgraph
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

 4571288 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 4661716 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 8791700 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 3173254 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 2648697 Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 3176517 Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
  583599 Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
  242390 Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
  153392 Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed


In [65]:
%%bash

for f in *bedgraph-Meth
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

  450582 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  528902 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
 1059904 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  257741 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  184742 Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  231347 Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  106695 Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
   45506 Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
   29468 Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed


In [66]:
%%bash

for f in *bedgraph-sparseMeth
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

  547868 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
  517805 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
 1000337 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
  152042 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
  135052 Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
  179454 Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
   74839 Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
   28850 Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
   16793 Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed


In [67]:
%%bash

for f in *bedgraph-unMeth
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

 3572838 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 3615009 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 6731459 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 2763471 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 2328903 Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 2765716 Meth15_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
  402065 Meth16_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
  168034 Meth17_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
  107131 Meth18_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed


In [68]:
#Confirm BEDfile creation
!find *.bed

Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
Meth14_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
Meth14_R1_001_val_1

In [69]:
#Confirm file creation
!head Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed

1	6570	6572
1	6713	6715
1	6780	6782
1	6813	6815
1	6818	6820
1	27606	27608
1	27613	27615
1	27641	27643
1	27643	27645
1	27674	27676


#### 4b. Genes

In [70]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -u \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.gene.gff \
  > ${f}-mcGenes
done

In [71]:
#Check output
!head *mcGenes

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcGenes <==
1	58745	58747
1	103334	103336
1	103347	103349
1	103356	103358
1	103360	103362
1	103398	103400
1	105953	105955
1	106012	106014
1	106155	106157
1	106173	106175

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcGenes <==
1	80133	80135
1	106202	106204
1	184227	184229
1	184266	184268
1	184271	184273
1	238091	238093
1	307885	307887
1	323373	323375
1	324004	324006
1	324495	324497

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcGenes <==
1	42893	42895
1	42959	42961
1	42970	42972
1	42977	42979
1	43005	43007
1	75959	75961
1	75962	75964
1	75993	75995
1	75996	75998
1	80015	80017

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcGenes <==
1	42893	42895
1	42959	42961
1	42970	42972
1	42977	42979
1	43005	43007
1	58745	58747
1	75959	75961
1	75962	75964
1	75993	75995
1	75996	75998

==> Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcGenes <==
1	58745	58747
1	59207	59209
1	69

In [72]:
#Count number of overlaps
!wc -l *mcGenes

  230343 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcGenes
  196827 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcGenes
 1255899 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcGenes
 1683069 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcGenes
  267181 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcGenes
  188348 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcGenes
 1284620 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcGenes
 1740149 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcGenes
  533230 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcGenes
  348967 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcGenes
 2322688 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcGenes
 3204885 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcGenes
  123271 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcGenes
   53235 Meth13_R1_001_val_1_

In [73]:
!wc -l *mcGenes > Mcap-5x-mcGenes-counts.txt

#### 4c. Coding Sequences (CDS)

In [74]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -u \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.CDS.gff \
  > ${f}-mcCDS
done

In [75]:
#Check output
!head *mcCDS

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcCDS <==
1	58745	58747
1	438779	438781
1	438791	438793
1	786125	786127
1	786144	786146
1	789544	789546
1	789590	789592
1	879226	879228
1	983540	983542
1	1263116	1263118

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcCDS <==
1	184266	184268
1	184271	184273
1	238091	238093
1	307885	307887
1	345002	345004
1	349163	349165
1	356955	356957
1	401872	401874
1	401879	401881
1	402084	402086

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcCDS <==
1	75959	75961
1	75962	75964
1	75993	75995
1	75996	75998
1	80478	80480
1	109952	109954
1	109984	109986
1	133020	133022
1	133022	133024
1	133024	133026

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcCDS <==
1	58745	58747
1	75959	75961
1	75962	75964
1	75993	75995
1	75996	75998
1	80478	80480
1	109952	109954
1	109984	109986
1	133020	133022
1	133022	133024

==> Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcCDS <==
1	58745	58747
1	5920

In [76]:
#Count number of overlaps
!wc -l *mcCDS

   54412 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcCDS
   60266 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcCDS
  361901 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcCDS
  476579 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcCDS
   64070 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcCDS
   58258 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcCDS
  374559 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcCDS
  496887 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcCDS
  113396 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcCDS
   89455 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcCDS
  589114 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcCDS
  791965 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcCDS
   18158 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcCDS
   11383 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgrap

In [77]:
!wc -l *mcCDS > Mcap-5x-mcCDS-counts.txt

#### 4d. Introns

In [78]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -u \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.intron.gff \
  > ${f}-mcIntrons
done

In [79]:
#Check output
!head *mcIntrons

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntrons <==
1	103334	103336
1	103347	103349
1	103356	103358
1	103360	103362
1	103398	103400
1	105953	105955
1	106012	106014
1	106155	106157
1	106173	106175
1	106216	106218

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcIntrons <==
1	80133	80135
1	106202	106204
1	184227	184229
1	323373	323375
1	324004	324006
1	324495	324497
1	324782	324784
1	327501	327503
1	327818	327820
1	331324	331326

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcIntrons <==
1	42893	42895
1	42959	42961
1	42970	42972
1	42977	42979
1	43005	43007
1	80015	80017
1	80040	80042
1	80077	80079
1	80101	80103
1	80107	80109

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcIntrons <==
1	42893	42895
1	42959	42961
1	42970	42972
1	42977	42979
1	43005	43007
1	80015	80017
1	80040	80042
1	80077	80079
1	80101	80103
1	80107	80109

==> Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntrons <==
1	69235	69237
1	6927

In [80]:
#Count number of overlaps
!wc -l *mcIntrons

  176102 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntrons
  136671 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcIntrons
  894836 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcIntrons
 1207609 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcIntrons
  203311 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntrons
  130195 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcIntrons
  910897 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcIntrons
 1244403 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcIntrons
  420249 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntrons
  259702 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcIntrons
 1735085 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcIntrons
 2415036 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcIntrons
  105145 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntrons
   

In [81]:
!wc -l *mcIntrons > Mcap-5x-mcIntrons-counts.txt

#### 4e. Flanking regions

In [82]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -u \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.flanks.gff \
  > ${f}-mcFlanks
done

In [83]:
#Check output
!head *mcFlanks

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcFlanks <==
1	443126	443128
1	444404	444406
1	1392759	1392761
1	1392780	1392782
1	1392793	1392795
1	1392832	1392834
1	1392838	1392840
1	1392908	1392910
1	1392921	1392923
1	1396199	1396201

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcFlanks <==
1	27782	27784
1	148080	148082
1	150099	150101
1	182756	182758
1	185808	185810
1	185814	185816
1	185830	185832
1	185844	185846
1	185868	185870
1	185879	185881

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcFlanks <==
1	27606	27608
1	27613	27615
1	27641	27643
1	27643	27645
1	27674	27676
1	27718	27720
1	27741	27743
1	27758	27760
1	27760	27762
1	27768	27770

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcFlanks <==
1	27606	27608
1	27613	27615
1	27641	27643
1	27643	27645
1	27674	27676
1	27718	27720
1	27741	27743
1	27758	27760
1	27760	27762
1	27768	27770

==> Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcFlanks <==
1	443126	

In [84]:
#Count number of overlaps
!wc -l *mcFlanks

   42417 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcFlanks
   59006 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcFlanks
  358799 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcFlanks
  460222 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcFlanks
   49804 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcFlanks
   55242 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcFlanks
  364811 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcFlanks
  469857 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcFlanks
  102126 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcFlanks
  104542 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcFlanks
  680960 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcFlanks
  887628 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcFlanks
   25660 Meth13_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcFlanks
   15818 Meth13_

In [85]:
!wc -l *mcFlanks > Mcap-5x-mcFlanks-counts.txt

#### 4f. Upstream flanking regions

In [ ]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -u \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.flanks.Upstream.gff \
  > ${f}-mcFlanksUpstream
done

In [ ]:
#Check output
!head *mcFlanksUpstream

In [ ]:
#Count number of overlaps
!wc -l *mcFlanksUpstream

In [ ]:
!wc -l *mcFlanksUpstream > Mcap-5x-mcFlanksUpstream-counts.txt

#### 4g. Downstream flanking regions

In [ ]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -u \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.flanks.Downstream.gff \
  > ${f}-mcFlanksUpstream
done

In [ ]:
#Check output
!head *mcFlanksDownstream

In [ ]:
#Count number of overlaps
!wc -l *mcFlanksDownstream

In [ ]:
!wc -l *mcFlanksDownstream > Mcap-5x-mcFlanksDownstream-counts.txt

#### 4h. Intergenic

In [72]:
%%bash 

for f in *bed
do
  /usr/local/bin/intersectBed \
  -u \
  -a ${f} \
  -b ../../../genome-feature-files/Mcap.GFFannotation.intergenic.bed \
  > ${f}-mcIntergenic
done

In [73]:
#Check output
!head *mcIntergenic

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntergenic <==
1	320600	320602
1	320631	320633
1	443126	443128
1	444404	444406
1	446577	446579
1	446641	446643
1	446659	446661
1	446682	446684
1	446691	446693
1	446746	446748

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcIntergenic <==
1	27782	27784
1	140551	140553
1	148080	148082
1	150099	150101
1	169735	169737
1	169771	169773
1	169796	169798
1	169800	169802
1	182756	182758
1	185808	185810

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcIntergenic <==
1	6570	6572
1	6713	6715
1	6780	6782
1	6813	6815
1	6818	6820
1	27606	27608
1	27613	27615
1	27641	27643
1	27643	27645
1	27674	27676

==> Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcIntergenic <==
1	6570	6572
1	6713	6715
1	6780	6782
1	6813	6815
1	6818	6820
1	27606	27608
1	27613	27615
1	27641	27643
1	27643	27645
1	27674	27676

==> Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntergenic <==
1	6905	6907
1	7273	7275
1

In [74]:
#Count number of overlaps
!wc -l *mcIntergenic

  220239 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntergenic
  351041 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcIntergenic
 2316939 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcIntergenic
 2888219 Meth10_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcIntergenic
  261721 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntergenic
  329457 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcIntergenic
 2330389 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcIntergenic
 2921567 Meth11_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcIntergenic
  526674 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-mcIntergenic
  651370 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-mcIntergenic
 4408771 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-mcIntergenic
 5586815 Meth12_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-mcIntergenic
  134470 Meth13_R1_001_val_1_bismark_bt2_pe

In [14]:
!wc -l *mcIntergenic > Mcap-5x-mcIntergenic-counts.txt

## *P. acuta*

In [14]:
cd ..

/Users/yaamini/Documents/Meth_Compare/analyses/Characterizing-CpG-Methylation-5x


In [15]:
#Make a directory for Pact output
#!mkdir Pact

In [16]:
cd Pact/

/Users/yaamini/Documents/Meth_Compare/analyses/Characterizing-CpG-Methylation-5x/Pact


### 1. Characterize CG motif locations in feature tracks

#### 1a. Set variable paths

In [23]:
paGenes = "../../../genome-feature-files/Pact.GFFannotation.Genes.gff"

In [24]:
paCDS = "../../../genome-feature-files/Pact.GFFannotation.CDS.gff"

In [25]:
paIntron = "../../../genome-feature-files/Pact.GFFannotation.Intron.gff"

In [26]:
paFlanks = "../../../genome-feature-files/Pact.GFFannotation.flanks.gff"

In [19]:
paUpstream = "../../../genome-feature-files/Pact.GFFannotation.flanks.Upstream.gff"

In [20]:
paDownstream = "../../../genome-feature-files/Pact.GFFannotation.flanks.Downstream.gff"

In [17]:
paIntergenic = "../../../genome-feature-files/Pact.GFFannotation.intergenic.bed"

In [18]:
paCGMotifs = "../../../genome-feature-files/Pact_CpG.gff"

#### 1b. Check variable paths

In [22]:
!head {paGenes}
!wc -l {paGenes}

scaffold6_cov64	AUGUSTUS	gene	1	5652	0.46	-	.	g1
scaffold6_cov64	AUGUSTUS	gene	5805	6678	0.57	+	.	g2
scaffold7_cov100	AUGUSTUS	gene	1	2566	0.96	+	.	g3
scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	AUGUSTUS	gene	7069	9073	1	-	.	g5
scaffold7_cov100	AUGUSTUS	gene	9590	11670	0.8	-	.	g6
scaffold7_cov100	AUGUSTUS	gene	13339	15463	0.92	-	.	g7
scaffold7_cov100	AUGUSTUS	gene	15738	18320	0.96	+	.	g8
scaffold7_cov100	AUGUSTUS	gene	18586	19270	0.99	-	.	g9
scaffold7_cov100	AUGUSTUS	gene	19312	20050	0.74	+	.	g10
   64558 ../../../genome-feature-files/Pact.GFFannotation.Genes.gff


In [23]:
!head {paCDS}
!wc -l {paCDS}

scaffold6_cov64	AUGUSTUS	CDS	495	842	0.84	-	2	transcript_id "g1.t1"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	CDS	1208	1555	0.92	-	2	transcript_id "g1.t1"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	CDS	1922	2269	1	-	2	transcript_id "g1.t1"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	CDS	5583	5652	0.26	-	0	transcript_id "g1.t1"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	CDS	495	842	0.84	-	2	transcript_id "g1.t2"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	CDS	1208	1555	0.92	-	2	transcript_id "g1.t2"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	CDS	1922	2269	1	-	2	transcript_id "g1.t2"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	CDS	4754	4851	0.4	-	1	transcript_id "g1.t2"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	CDS	5594	5652	0.54	-	0	transcript_id "g1.t2"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	CDS	5805	5838	0.98	+	0	transcript_id "g2.t1"; gene_id "g2";
  318484 ../../../genome-feature-files/Pact.GFFannotation.CDS.gff


In [24]:
!head {paIntron}
!wc -l {paIntron}

scaffold6_cov64	AUGUSTUS	intron	1	494	0.82	-	.	transcript_id "g1.t1"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	intron	843	1207	0.92	-	.	transcript_id "g1.t1"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	intron	1556	1921	1	-	.	transcript_id "g1.t1"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	intron	2270	5582	0.23	-	.	transcript_id "g1.t1"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	intron	1	494	0.82	-	.	transcript_id "g1.t2"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	intron	843	1207	0.92	-	.	transcript_id "g1.t2"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	intron	1556	1921	1	-	.	transcript_id "g1.t2"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	intron	2270	4753	0.4	-	.	transcript_id "g1.t2"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	intron	4852	5593	0.48	-	.	transcript_id "g1.t2"; gene_id "g1";
scaffold6_cov64	AUGUSTUS	intron	5839	5945	0.54	+	.	transcript_id "g2.t1"; gene_id "g2";
  241534 ../../../genome-feature-files/Pact.GFFannotation.Intron.gff


In [32]:
!head {paFlanks}
!wc -l {paFlanks}

scaffold6_cov64	AUGUSTUS	gene	5653	5804	0.46	-	.	g1
scaffold6_cov64	AUGUSTUS	gene	5653	5804	0.57	+	.	g2
scaffold6_cov64	AUGUSTUS	gene	6679	7678	0.57	+	.	g2
scaffold7_cov100	AUGUSTUS	gene	2567	3466	0.96	+	.	g3
scaffold7_cov100	AUGUSTUS	gene	2567	3466	0.78	-	.	g4
scaffold7_cov100	AUGUSTUS	gene	6218	7068	0.78	-	.	g4
scaffold7_cov100	AUGUSTUS	gene	6218	7068	1	-	.	g5
scaffold7_cov100	AUGUSTUS	gene	9074	9589	1	-	.	g5
scaffold7_cov100	AUGUSTUS	gene	9074	9589	0.8	-	.	g6
scaffold7_cov100	AUGUSTUS	gene	11671	12670	0.8	-	.	g6
  143874 ../../../genome-feature-files/Pact.GFFannotation.flanks.gff


In [22]:
!head {paUpstream}
!wc -l {paUpstream}

scaffold6_cov64	AUGUSTUS	gene	5653	5804	0.46	-	.	g1
scaffold6_cov64	AUGUSTUS	gene	5653	5804	0.57	+	.	g2
scaffold7_cov100	AUGUSTUS	gene	6218	7068	0.78	-	.	g4
scaffold7_cov100	AUGUSTUS	gene	9074	9589	1	-	.	g5
scaffold7_cov100	AUGUSTUS	gene	11671	12670	0.8	-	.	g6
scaffold7_cov100	AUGUSTUS	gene	15464	15737	0.92	-	.	g7
scaffold7_cov100	AUGUSTUS	gene	15464	15737	0.96	+	.	g8
scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.99	-	.	g9
scaffold7_cov100	AUGUSTUS	gene	20051	20077	0.99	-	.	g9
scaffold7_cov100	AUGUSTUS	gene	18321	18585	0.74	+	.	g10
   70639 ../../../genome-feature-files/Pact.GFFannotation.flanks.Upstream.gff


In [23]:
!head {paDownstream}
!wc -l {paDownstream}

scaffold6_cov64	AUGUSTUS	gene	1	0	0.46	-	.	g1
scaffold6_cov64	AUGUSTUS	gene	6679	7678	0.57	+	.	g2
scaffold7_cov100	AUGUSTUS	gene	2567	3466	0.96	+	.	g3
scaffold7_cov100	AUGUSTUS	gene	2567	3466	0.78	-	.	g4
scaffold7_cov100	AUGUSTUS	gene	6218	7068	1	-	.	g5
scaffold7_cov100	AUGUSTUS	gene	9074	9589	0.8	-	.	g6
scaffold7_cov100	AUGUSTUS	gene	12339	13338	0.92	-	.	g7
scaffold7_cov100	AUGUSTUS	gene	18321	18585	0.96	+	.	g8
scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.96	+	.	g8
scaffold7_cov100	AUGUSTUS	gene	18321	18585	0.99	-	.	g9
   73996 ../../../genome-feature-files/Pact.GFFannotation.flanks.Downstream.gff


In [19]:
!head {paIntergenic}
!wc -l {paIntergenic}

scaffold1_cov55	0	421
scaffold2_cov51	0	1151
scaffold3_cov83	0	598
scaffold4_cov57	0	192
scaffold5_cov26	0	102
scaffold6_cov64	7678	8236
scaffold7_cov100	25295	27516
scaffold7_cov100	30779	30897
scaffold7_cov100	38761	40187
scaffold7_cov100	83819	86977
  185643 ../../../genome-feature-files/Pact.GFFannotation.intergenic.bed


In [28]:
!head {paCGMotifs}
!wc -l {paCGMotifs}

##gff-version 2.0
##date 2020-03-29
##Type DNA scaffold1_cov55
scaffold1_cov55	fuzznuc	misc_feature	23	24	2.000	+	.	Sequence "scaffold1_cov55.1" ; note "*pat pattern1"
scaffold1_cov55	fuzznuc	misc_feature	35	36	2.000	+	.	Sequence "scaffold1_cov55.2" ; note "*pat pattern1"
scaffold1_cov55	fuzznuc	misc_feature	50	51	2.000	+	.	Sequence "scaffold1_cov55.3" ; note "*pat pattern1"
scaffold1_cov55	fuzznuc	misc_feature	85	86	2.000	+	.	Sequence "scaffold1_cov55.4" ; note "*pat pattern1"
scaffold1_cov55	fuzznuc	misc_feature	93	94	2.000	+	.	Sequence "scaffold1_cov55.5" ; note "*pat pattern1"
scaffold1_cov55	fuzznuc	misc_feature	103	104	2.000	+	.	Sequence "scaffold1_cov55.6" ; note "*pat pattern1"
scaffold1_cov55	fuzznuc	misc_feature	106	107	2.000	+	.	Sequence "scaffold1_cov55.7" ; note "*pat pattern1"
 9639415 ../../../genome-feature-files/Pact_CpG.gff


#### 1c. Characterize overlaps with `bedtools`

In [29]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {paCGMotifs} \
-b {paGenes} \
> Pact-CGMotif-Gene-Overlaps.txt

In [30]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {paCGMotifs} \
-b {paCDS} \
> Pact-CGMotif-CDS-Overlaps.txt

In [31]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {paCGMotifs} \
-b {paIntron} \
> Pact-CGMotif-Intron-Overlaps.txt

In [32]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {paCGMotifs} \
-b {paFlanks} \
> Pact-CGMotif-Flanks-Overlaps.txt

In [31]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {paCGMotifs} \
-b {paUpstream} \
> Pact-CGMotif-Flanks-Upstream-Overlaps.txt

In [33]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {paCGMotifs} \
-b {paDownstream} \
> Pact-CGMotif-Flanks-Downstream-Overlaps.txt

In [20]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {paCGMotifs} \
-b {paIntergenic} \
> Pact-CGMotif-Intergenic-Overlaps.txt

In [22]:
!wc -l *CGMotif* > Pact-CGMotif-Overlaps-counts.txt

#### 1d. Summary

| *P. acuta* Genome Feature 	| **Number individual features** 	| **Overlaps with CG Motifs** 	| **% Total CG Motifs** 	|
|:-------------------------------:	|:------------------------------:	|:---------------------------:	|:---------------------:	|
|              Genes              	|              64558             	|           3434720           	|          35.6         	|
|         Coding Sequences        	|             318484             	|           1455630           	|          15.1         	|
|             Introns             	|             241534             	|           1999490           	|          20.7         	|
|             Flanking Regions             	|             143874             	|           1732726           	|          17.8         	|
|             Upstream Flanks             	|             70639             	|           1047316           	|          10.9         	|
|             Downstream Flanks             	|             73996             	|           948914           	|          9.8         	|
|        Intergenic Regions       	|               185643              	|           3989278           	|          41.4         	|

### 2. Download coverage files

In [34]:
#Download Pact WGBS and MBD-BS 5x sample bedgraphs
!wget -r -l1 --no-parent -A "*5x.bedgraph" https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Pact_tg/dedup/

--2020-05-05 13:25:32--  https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Pact_tg/dedup/
Resolving gannet.fish.washington.edu (gannet.fish.washington.edu)... 128.95.149.52
Connecting to gannet.fish.washington.edu (gannet.fish.washington.edu)|128.95.149.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Pact_tg/dedup/index.html.tmp’

gannet.fish.washing     [ <=>                ]  42.11K  --.-KB/s    in 0.001s  

2020-05-05 13:25:34 (47.9 MB/s) - ‘gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Pact_tg/dedup/index.html.tmp’ saved [43123]

Loading robots.txt; please ignore errors.
--2020-05-05 13:25:34--  https://gannet.fish.washington.edu/robots.txt
Reusing existing connection to gannet.fish.washington.edu:443.
HTTP request sent, awaiting response... 404 Not Found
2020-05-05 13:25:34 ERROR 404: Not Found.

Removing gannet

In [35]:
#Move samples from directory structure on gannet to cd
!mv gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Pact_tg/dedup/* .

In [36]:
#Remove empty directory
!rm -r gannet.fish.washington.edu/

In [37]:
#Check files
!find *bedgraph

Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph


In [38]:
#Download Pact RRBS 5x sample bedgraphs
!wget -r -l1 --no-parent -A "*5x.bedgraph" https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Pact_tg/nodedup/

--2020-05-05 13:25:49--  https://gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Pact_tg/nodedup/
Resolving gannet.fish.washington.edu (gannet.fish.washington.edu)... 128.95.149.52
Connecting to gannet.fish.washington.edu (gannet.fish.washington.edu)|128.95.149.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Pact_tg/nodedup/index.html.tmp’

gannet.fish.washing     [ <=>                ]  19.51K  --.-KB/s    in 0.001s  

2020-05-05 13:25:50 (32.7 MB/s) - ‘gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Pact_tg/nodedup/index.html.tmp’ saved [19983]

Loading robots.txt; please ignore errors.
--2020-05-05 13:25:50--  https://gannet.fish.washington.edu/robots.txt
Reusing existing connection to gannet.fish.washington.edu:443.
HTTP request sent, awaiting response... 404 Not Found
2020-05-05 13:25:50 ERROR 404: Not Found.

Removing 

In [39]:
#Move samples from directory structure on gannet to cd
!mv gannet.fish.washington.edu/seashell/bu-mox/scrubbed/031520-TG-bs/Pact_tg/nodedup/* .

In [40]:
#Remove empty directory
!rm -r gannet.fish.washington.edu/

In [41]:
!find *bedgraph

Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph


In [42]:
#Verify checksums from gannet
!md5sum -c ../Pact-5xbedgraph-GANNET-md5sum.txt

Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK
Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph: OK


In [44]:
!wc -l *bedgraph

 5546051 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 6358722 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 5866786 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 1835561 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 1451229 Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 1517358 Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 2640625 Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
  539008 Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 2732607 Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph
 28487947 total


In [18]:
!wc -l *bedgraph > Pact-5x-bedgraph-counts.txt

### 3. Characterize methylation for each CpG dinucleotide

- Methylated: > 50% methylation
- Sparsely methylated: 10-50% methylation
- Unmethylated: < 10% methylation

##### Methylated loci

In [45]:
%%bash
for f in *bedgraph
do
    awk '{if ($4 >= 50) { print $1, $2, $3, $4 }}' ${f} \
    > ${f}-Meth
done

In [46]:
!head *Meth

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth <==
scaffold7_cov100 4351 4353 50.000000
scaffold7_cov100 5500 5502 83.333333
scaffold7_cov100 5578 5580 57.142857
scaffold7_cov100 5986 5988 100.000000
scaffold7_cov100 6144 6146 100.000000
scaffold7_cov100 6188 6190 100.000000
scaffold7_cov100 6198 6200 88.888889
scaffold7_cov100 6231 6233 100.000000
scaffold7_cov100 6233 6235 100.000000
scaffold7_cov100 7438 7440 100.000000

==> Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth <==
scaffold7_cov100 5500 5502 62.500000
scaffold7_cov100 5986 5988 66.666667
scaffold7_cov100 6144 6146 100.000000
scaffold7_cov100 6188 6190 94.117647
scaffold7_cov100 6198 6200 100.000000
scaffold7_cov100 6231 6233 71.428571
scaffold7_cov100 6233 6235 100.000000
scaffold7_cov100 7438 7440 88.235294
scaffold7_cov100 7696 7698 95.833333
scaffold7_cov100 7796 7798 60.000000

==> Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth <==
scaffold7_cov100 5500 5502 87.500000
scaffold7_cov100 5578 5580 55.55

In [47]:
!wc -l *Meth

  110364 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  126440 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  124819 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
   31047 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
   30345 Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
   26617 Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  258222 Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  213342 Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
  255370 Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth
 1176566 total


In [19]:
!wc -l *-Meth > Pact-5x-Meth-counts.txt

##### Sparsely methylated loci

In [48]:
%%bash
for f in *bedgraph
do
    awk '{if ($4 < 50) { print $1, $2, $3, $4}}' ${f} \
    | awk '{if ($4 > 10) { print $1, $2, $3, $4 }}' \
    > ${f}-sparseMeth
done

In [49]:
!head *sparseMeth

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth <==
scaffold1_cov55 102 104 16.666667
scaffold1_cov55 186 188 20.000000
scaffold3_cov83 118 120 12.500000
scaffold3_cov83 137 139 12.500000
scaffold3_cov83 475 477 18.750000
scaffold3_cov83 484 486 14.893617
scaffold3_cov83 504 506 21.052632
scaffold6_cov64 7373 7375 12.500000
scaffold6_cov64 7983 7985 11.111111
scaffold7_cov100 1293 1295 11.111111

==> Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth <==
scaffold1_cov55 105 107 12.500000
scaffold1_cov55 252 254 20.000000
scaffold2_cov51 686 688 11.111111
scaffold6_cov64 3978 3980 11.111111
scaffold6_cov64 7077 7079 12.500000
scaffold7_cov100 2652 2654 16.666667
scaffold7_cov100 3994 3996 10.526316
scaffold7_cov100 7121 7123 25.000000
scaffold7_cov100 7201 7203 16.666667
scaffold7_cov100 10755 10757 13.333333

==> Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth <==
scaffold1_cov55 119 121 20.000000
scaffold1_cov55 194 196 20.000000
scaffold2_cov51 686 688 

In [50]:
!wc -l *sparseMeth

  367019 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  345887 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  385346 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  137700 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
   64837 Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
   89246 Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  296059 Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
   80086 Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
  337855 Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth
 2104035 total


In [20]:
!wc -l *-sparseMeth > Pact-5x-sparseMeth-counts.txt

##### Unmethylated loci

In [51]:
%%bash
for f in *bedgraph
do
    awk '{if ($4 <= 10) { print $1, $2, $3, $4 }}' ${f} \
    > ${f}-unMeth
done

In [52]:
!head *unMeth

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth <==
scaffold1_cov55 105 107 0.000000
scaffold1_cov55 116 118 0.000000
scaffold1_cov55 119 121 0.000000
scaffold1_cov55 146 148 0.000000
scaffold1_cov55 194 196 0.000000
scaffold2_cov51 649 651 0.000000
scaffold2_cov51 686 688 8.333333
scaffold2_cov51 778 780 0.000000
scaffold3_cov83 130 132 0.000000
scaffold3_cov83 189 191 6.250000

==> Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth <==
scaffold1_cov55 49 51 0.000000
scaffold1_cov55 84 86 0.000000
scaffold1_cov55 92 94 0.000000
scaffold1_cov55 102 104 0.000000
scaffold1_cov55 116 118 0.000000
scaffold1_cov55 119 121 0.000000
scaffold1_cov55 146 148 0.000000
scaffold1_cov55 169 171 0.000000
scaffold1_cov55 186 188 0.000000
scaffold1_cov55 194 196 0.000000

==> Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth <==
scaffold1_cov55 250 252 0.000000
scaffold2_cov51 649 651 0.000000
scaffold2_cov51 778 780 0.000000
scaffold3_cov83 118 120 0.000000
scaffold3_cov83 130 132 0.

In [53]:
!wc -l *unMeth

 5068668 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5886395 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 5356621 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 1666814 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 1356047 Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 1401495 Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 2086344 Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
  245580 Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 2139382 Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth
 25207346 total


In [21]:
!wc -l *-unMeth > Pact-5x-unMeth-counts.txt

### 4. Characterize genomic locations of CpGs

#### 4a. Create BEDfiles

In [54]:
%%bash

for f in *bedgraph
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

 5546051 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 6358722 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 5866786 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 1835561 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 1451229 Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 1517358 Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 2640625 Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
  539008 Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
 2732607 Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed


In [55]:
%%bash

for f in *bedgraph-Meth
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

  110364 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  126440 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  124819 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
   31047 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
   30345 Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
   26617 Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  258222 Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  213342 Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
  255370 Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed


In [56]:
%%bash

for f in *bedgraph-sparseMeth
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

  367019 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
  345887 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
  385346 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
  137700 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
   64837 Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
   89246 Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
  296059 Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
   80086 Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
  337855 Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed


In [57]:
%%bash

for f in *bedgraph-unMeth
do
    awk '{print $1"\t"$2"\t"$3}' ${f} > ${f}.bed
    wc -l ${f}.bed
done

 5068668 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 5886395 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 5356621 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 1666814 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 1356047 Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 1401495 Meth6_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 2086344 Meth7_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
  245580 Meth8_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
 2139382 Meth9_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed


In [58]:
#Confirm BEDfile creation
!find *.bed

Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed
Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed
Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed
Meth5_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed
Meth5_R1_001_val_1_bismark_bt2_pe._5

In [60]:
#Confirm file creation
!head Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed

scaffold1_cov55	102	104
scaffold1_cov55	105	107
scaffold1_cov55	116	118
scaffold1_cov55	119	121
scaffold1_cov55	146	148
scaffold1_cov55	186	188
scaffold1_cov55	194	196
scaffold2_cov51	649	651
scaffold2_cov51	686	688
scaffold2_cov51	778	780


#### 4b. Genes

In [57]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.Genes.gff \
  > ${f}-paGenes
done

In [58]:
#Check output
!head *paGenes

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paGenes <==
scaffold7_cov100	4351	4353	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	5500	5502	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	5578	5580	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	5986	5988	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	6144	6146	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	6188	6190	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	6198	6200	scaffold7_cov100	AUGUSTUS	gene	3467	6217	0.78	-	.	g4
scaffold7_cov100	7438	7440	scaffold7_cov100	AUGUSTUS	gene	7069	9073	1	-	.	g5
scaffold7_cov100	7696	7698	scaffold7_cov100	AUGUSTUS	gene	7069	9073	1	-	.	g5
scaffold7_cov100	7796	7798	scaffold7_cov100	AUGUSTUS	gene	7069	9073	1	-	.	g5

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paGenes <==
scaffold7_cov100	1293	1295	scaffold7_cov100	AUGUSTUS	gene	

In [59]:
#Count number of overlaps
!wc -l *paGenes

   73959 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paGenes
  157337 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paGenes
 2235696 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paGenes
 2466992 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paGenes
   85861 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paGenes
  144292 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paGenes
 2531803 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paGenes
 2761956 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paGenes
   82377 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paGenes
  161791 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paGenes
 2344110 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paGenes
 2588278 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paGenes
   13588 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paGenes
   56290 Meth4_R1_001_val_1_bismark_bt2_pe

In [22]:
!wc -l *paGenes > Pact-5x-paGenes-counts.txt

#### 4c. Coding Sequences (CDS)

In [60]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.CDS.gff \
  > ${f}-paCDS
done

In [61]:
#Check output
!head *paCDS

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paCDS <==
scaffold7_cov100	5500	5502	scaffold7_cov100	AUGUSTUS	CDS	5466	5540	1	-	2	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	5500	5502	scaffold7_cov100	AUGUSTUS	CDS	5466	5540	1	-	2	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	6144	6146	scaffold7_cov100	AUGUSTUS	CDS	6091	6217	0.48	-	0	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	6144	6146	scaffold7_cov100	AUGUSTUS	CDS	6091	6211	0.52	-	0	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	6188	6190	scaffold7_cov100	AUGUSTUS	CDS	6091	6217	0.48	-	0	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	6188	6190	scaffold7_cov100	AUGUSTUS	CDS	6091	6211	0.52	-	0	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	6198	6200	scaffold7_cov100	AUGUSTUS	CDS	6091	6217	0.48	-	0	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	6198	6200	scaffold7_cov100	AUGUSTUS	CDS	6091	6211	0.52	-	0	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	7696	7698	scaff

In [62]:
#Count number of overlaps
!wc -l *paCDS

   59188 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paCDS
   89863 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paCDS
 1345289 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paCDS
 1494340 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paCDS
   66365 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paCDS
   76868 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paCDS
 1477399 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paCDS
 1620632 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paCDS
   65245 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paCDS
   89654 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paCDS
 1397816 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paCDS
 1552715 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paCDS
    9644 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paCDS
   36616 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.b

In [23]:
!wc -l *paCDS > Pact-5x-paCDS-counts.txt

#### 4d. Introns

In [63]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.Intron.gff \
  > ${f}-paIntron
done

In [64]:
#Check output
!head *paIntron

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paIntron <==
scaffold7_cov100	4351	4353	scaffold7_cov100	AUGUSTUS	intron	4181	4607	1	-	.	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	4351	4353	scaffold7_cov100	AUGUSTUS	intron	4181	4607	1	-	.	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	5578	5580	scaffold7_cov100	AUGUSTUS	intron	5541	6090	1	-	.	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	5578	5580	scaffold7_cov100	AUGUSTUS	intron	5541	6090	1	-	.	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	5986	5988	scaffold7_cov100	AUGUSTUS	intron	5541	6090	1	-	.	transcript_id "g4.t1"; gene_id "g4";
scaffold7_cov100	5986	5988	scaffold7_cov100	AUGUSTUS	intron	5541	6090	1	-	.	transcript_id "g4.t2"; gene_id "g4";
scaffold7_cov100	7438	7440	scaffold7_cov100	AUGUSTUS	intron	7104	7649	1	-	.	transcript_id "g5.t1"; gene_id "g5";
scaffold7_cov100	7438	7440	scaffold7_cov100	AUGUSTUS	intron	7104	7649	1	-	.	transcript_id "g5.t2"; gene_id "g5";
scaffold7_cov100	7796	7

In [65]:
#Count number of overlaps
!wc -l *paIntron

   41787 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paIntron
  122271 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paIntron
 1676080 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paIntron
 1840138 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paIntron
   51567 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paIntron
  117738 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paIntron
 1943990 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paIntron
 2113295 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paIntron
   47352 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paIntron
  128122 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paIntron
 1770676 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paIntron
 1946150 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paIntron
    8446 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paIntron
   38589 Meth4_R1_001_val_1_b

In [24]:
!wc -l *paIntron > Pact-5x-paIntron-counts.txt

#### 4e. Flanking regions

In [61]:
%%bash

for f in *bed
do
  /usr/local/bin/intersectBed \
  -wb \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.flanks.gff \
  > ${f}-paFlanks
done

In [62]:
#Check output
!head *paFlanks

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paFlanks <==
scaffold7_cov100	6231	6233	scaffold7_cov100	AUGUSTUS	gene	6218	7068	0.78	-	.	g4
scaffold7_cov100	6231	6233	scaffold7_cov100	AUGUSTUS	gene	6218	7068	1	-	.	g5
scaffold7_cov100	6233	6235	scaffold7_cov100	AUGUSTUS	gene	6218	7068	0.78	-	.	g4
scaffold7_cov100	6233	6235	scaffold7_cov100	AUGUSTUS	gene	6218	7068	1	-	.	g5
scaffold7_cov100	19284	19286	scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.96	+	.	g8
scaffold7_cov100	19284	19286	scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.99	-	.	g9
scaffold7_cov100	19284	19286	scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.74	+	.	g10
scaffold7_cov100	19284	19286	scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.39	+	.	g11
scaffold7_cov100	19296	19298	scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.96	+	.	g8
scaffold7_cov100	19296	19298	scaffold7_cov100	AUGUSTUS	gene	19271	19311	0.99	-	.	g9

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paFlanks <==
scaffold6_cov64	7373	7375	s

In [63]:
#Count number of overlaps
!wc -l *paFlanks

   28031 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paFlanks
   97808 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paFlanks
 1317046 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paFlanks
 1442885 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paFlanks
   32259 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paFlanks
   93054 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paFlanks
 1536795 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paFlanks
 1662108 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paFlanks
   31840 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paFlanks
  102280 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paFlanks
 1392903 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paFlanks
 1527023 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paFlanks
    7491 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paFlanks
   34079 Meth4_R1_001_val_1_b

In [64]:
!wc -l *paFlanks > Pact-5x-paFlanks-counts.txt

#### 4e. Intergenic

In [66]:
%%bash 

for f in *bed
do
  /usr/local/bin/intersectBed \
  -v \
  -a ${f} \
  -b ../../../genome-feature-files/Pact.GFFannotation.Genes.gff \
  > ${f}-paIntergenic
done

In [67]:
#Check output
!head *paIntergenic

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paIntergenic <==
scaffold7_cov100	6231	6233
scaffold7_cov100	6233	6235
scaffold7_cov100	19284	19286
scaffold7_cov100	19296	19298
scaffold7_cov100	24494	24496
scaffold7_cov100	24509	24511
scaffold7_cov100	24557	24559
scaffold7_cov100	24617	24619
scaffold7_cov100	24895	24897
scaffold7_cov100	24941	24943

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paIntergenic <==
scaffold1_cov55	102	104
scaffold1_cov55	186	188
scaffold3_cov83	118	120
scaffold3_cov83	137	139
scaffold3_cov83	475	477
scaffold3_cov83	484	486
scaffold3_cov83	504	506
scaffold6_cov64	7373	7375
scaffold6_cov64	7983	7985
scaffold7_cov100	13275	13277

==> Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paIntergenic <==
scaffold1_cov55	105	107
scaffold1_cov55	116	118
scaffold1_cov55	119	121
scaffold1_cov55	146	148
scaffold1_cov55	194	196
scaffold2_cov51	649	651
scaffold2_cov51	686	688
scaffold2_cov51	778	780
scaffold3_cov83	130	132
scaffold

In [68]:
#Count number of overlaps
!wc -l *paIntergenic

   36461 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paIntergenic
  209781 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paIntergenic
 2834593 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paIntergenic
 3080835 Meth1_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paIntergenic
   40642 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paIntergenic
  201712 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paIntergenic
 3356494 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paIntergenic
 3598848 Meth2_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paIntergenic
   42507 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-Meth.bed-paIntergenic
  223666 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-sparseMeth.bed-paIntergenic
 3014184 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph-unMeth.bed-paIntergenic
 3280357 Meth3_R1_001_val_1_bismark_bt2_pe._5x.bedgraph.bed-paIntergenic
   17473 Meth4_R1_001_val_1_bismark_bt2_pe._5x.bedgraph

In [25]:
!wc -l *paIntergenic > Pact-5x-paIntergenic-counts.txt